# $\textbf{Linear Feedback Shift Registers (LFSRs)}$
$\text{Author: Ryan Burns}$

$\text{For background on LFSRs and maximum-length }m\text{-sequences, visit: }\textit{https://en.wikipedia.org/wiki/Linear-feedback_shift_register.}$
$\text{It is assumed that a finite list of primitive polynomial coefficients (stored as hex values) are available for the polynomial degree specified below}$
$\text{via a locally stored file with name <degree>.txt. For degree-10 polynomials, for example, the corresponding primitive polynomial coefficients}$
$\text{would have file name 10.txt. LFSRs are built for each polynomial, each yielding a pseudorandom binary sequence of length/period }2^{\text{degree}}-1.$

### $\textbf{Import Dependencies}$

In [1]:
# System imports
from os import environ
from time import time

# So numpy and Tensorflow can cooperate
environ['KMP_DUPLICATE_LIB_OK']='True'

# Numpy imports
from numpy import array, matmul, transpose, identity, hstack, vstack, shape
from numpy import expand_dims, zeros, correlate, linspace, arange

# Galois / LFSR utilities
from galois_tools import *

# Plotting functionality
from matplotlib import pyplot as plt

### $\textbf{Specify Degree of Polynomial Over }GF(2)\textbf{ & Seed State for LFSR}$

In [2]:
# Primitive polynomial degree
deg = 10 # (= number of bits)

# Length of m-sequence generated
M = 2**deg - 1 # (bits)

# Initial state of register
seed = 1 # (decimal form)

### $\textbf{Load Primitive Polynomial Coefficients of Specified Degree Over }GF(2)$

In [3]:
# Read list of primitive polynomial coefficients for definition
# of the linear feedback shift registers yielding m-sequences
coeff_catalog = read_polynomials_from_file(deg=deg)

### $\textbf{Define Set of LFSRs of Specified Degree}$

In [4]:
LFSRs = [
    LFSR(mask=eval(hex2bin(coeff_catalog[n], nbits=deg)), 
    seed=seed, order=deg, register_id=str(n)) 
    for n in range(len(coeff_catalog))
]

### $\textbf{Generate Maximum Length Sequences (i.e., }m\textbf{-Sequences) With Each Register}$

In [5]:
# Maximimum length binary sequences of length M = 2^deg - 1 via LFSRs
m_sequences = vstack(tuple(register.m_sequence() for register in LFSRs))

# Convert m-sequences from logical {0,1} to algebraic {-1,+1} representation
algebraic_m_sequences = m_sequences.astype('float64') * 2 - 1

### $\textbf{Visualize Auto- & Cross Correlations of Algebraic } \pm1\textbf{ Representations}$

In [6]:
%matplotlib notebook

############################
# Correlation computations #
############################

# Correlation delays (bits/samples)
delays = arange(-M + 1, M)

# Autocorrelations, computed per algebraic m-sequence
autocorrelations = transpose([correlate(b, b, 'full')
    for b in algebraic_m_sequences])

# Cross-correlations of m-sequences 1 & 2 in collection
cross_corr_example = correlate(algebraic_m_sequences[0,:],
    algebraic_m_sequences[1,:],'full') 

######################################
# PLOT: m-sequence 1 autocorrelation #
######################################

# New figure
plt.figure(figsize=(9.9,6));

# Save these axes as ax
ax = plt.subplot(3,1,1);

# Add grid
plt.grid(c='k',alpha=0.25);

# Plot autocorrelation of m-sequence 1 of collection
plt.plot(delays,autocorrelations[:,0]/M,c='k',
    label='$m$-sequence length $M=2^m-1 = ' \
    + str(int(2**deg - 1)) \
    + ',\quad m = ' + str(deg) + '$');

# Set axis limits
plt.xlim([-M+1,M]);
plt.ylim([-0.1,1])

# Title
plt.title(r'Autocorrelation of m-sequence 1',weight='bold')

# Label axes
plt.ylabel(r'Normalized')
plt.xlabel(r'Delay (bits)')

# Legend
plt.legend()

######################################
# PLOT: m-sequence 2 autocorrelation #
######################################

# 2nd subplot on common axis
plt.subplot(3,1,2,sharex=ax);

# Add grid
plt.grid(c='k',alpha=0.25);

# Plot autocorrelation of m-sequence 2 of collection
plt.plot(delays,autocorrelations[:,1]/M,c='k',
    label='$m$-sequence length $M=2^m-1 = ' \
    + str(int(2**deg - 1)) \
    + ',\quad m = ' + str(deg) + '$');

# Set axis limits
plt.xlim([-M+1,M]);
plt.ylim([-0.1,1])

# Title
plt.title(r'Autocorrelation of m-sequence 2',weight='bold')

# Label axes
plt.ylabel(r'Normalized')
plt.xlabel(r'Delay (bits)')

# Legend
plt.legend()

############################################
# PLOT: m-sequence 1 & 2 cross-correlation #
############################################

# 3rd subplot on common axis
plt.subplot(3,1,3,sharex=ax)

# Add grid
plt.grid(c='k',alpha=0.25);

# Plot cross-correlation of m-sequences 1 and 2
plt.plot(delays,cross_corr_example/M,c='k',
    label='$m$-sequence length $M=2^m-1 = ' \
    + str(int(2**deg - 1)) \
    + ',\quad m = ' + str(deg) + '$');

# Set axis limits
plt.xlim([-M+1,M]);
plt.ylim([-0.1,1])

# Title
plt.title(r'Cross-correlation of m-sequences 1 and 2',weight='bold')

# Label axes
plt.ylabel(r'Normalized')
plt.xlabel(r'Delay (bits)')

# Legend
plt.legend()

# Optimize subplot layout
plt.tight_layout()

<IPython.core.display.Javascript object>

In [7]:
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Reshape, LSTM
from tensorflow.keras.models import Model;
from tensorflow.keras.optimizers import RMSprop;
from tensorflow.keras.initializers import Constant
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, AUC, Accuracy;
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives;

In [8]:
p = 9
N_epoch = 15000
pct_validation = 0
N_batch = 2**deg - 1
learning_rate = 0.001

In [9]:
# Maximimum length binary sequences of length M = 2^deg - 1 via LFSRs
b = vstack(tuple(register.stream(2**deg + deg) for register in LFSRs))
X = array([b[p,n:(n + deg)] for n in range(b.shape[1] - deg - 1)])
Y = array([b[p,(n + deg)] for n in range(b.shape[1] - deg - 1)])

In [10]:
x = Input(shape=(deg,))
h = Dense(deg,activation='sigmoid',use_bias=False)(x)
y = Dense(1,activation='sigmoid',use_bias=False)(h)
model = Model(x,y)

In [11]:
###########################
# Define loss & optimizer #
###########################

# Set RMSprop optimization for 
# speed-of-convergence purposes
opt = RMSprop(
    learning_rate=learning_rate,
    epsilon=1e-07,
    name="RMSprop"
);

# Model compilation, using categorical
# cross-entropy error w/ RMSprop
model.compile(
    
    # Error/loss function
    loss='binary_crossentropy', 
    
    # Use RMSprop
    optimizer=opt,
    
    # List metrics here
    metrics=[
        CategoricalAccuracy(),
        Accuracy(),
        TruePositives(),
        TrueNegatives(),
        FalsePositives(),
        FalseNegatives()
    ]
);

# Print a summary table
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                100       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 10        
Total params: 110
Trainable params: 110
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Model training
model.fit(
    
    # Input dataset
    x=X, # state vectot "observation" bits
    y=Y, # future-state predicted bits
    
    # Batch size
    batch_size=N_batch,
    
    # Number of training epochs
    epochs=N_epoch,
    
    # Print progress
    verbose=1,
    
    # Set aside fraction for validation
    validation_split=pct_validation,
    
    # False for time series
    shuffle=False,
    
    # Parallelize job across 2 workers
    workers=1,
    use_multiprocessing=False
);

Train on 1023 samples
Epoch 1/15000
1023/1023 [==============================] - 1s 1ms/sample - loss: 0.7050 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 9.0000 - true_negatives: 503.0000 - false_positives: 8.0000 - false_negatives: 503.0000
Epoch 2/15000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.7035 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 14.0000 - true_negatives: 499.0000 - false_positives: 12.0000 - false_negatives: 498.0000
Epoch 3/15000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.7026 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 20.0000 - true_negatives: 495.0000 - false_positives: 16.0000 - false_negatives: 492.0000
Epoch 4/15000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.7018 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 22.0000 - true_negatives: 488.0000 - false_positives: 23.0000 - false_

Epoch 34/15000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6955 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 174.0000 - true_negatives: 336.0000 - false_positives: 175.0000 - false_negatives: 338.0000
Epoch 35/15000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6955 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 179.0000 - true_negatives: 330.0000 - false_positives: 181.0000 - false_negatives: 333.0000
Epoch 36/15000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6954 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 181.0000 - true_negatives: 326.0000 - false_positives: 185.0000 - false_negatives: 331.0000
Epoch 37/15000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6954 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 186.0000 - true_negatives: 320.0000 - false_positives: 191.0000 - false_negative

Epoch 67/15000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6946 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 250.0000 - true_negatives: 261.0000 - false_positives: 250.0000 - false_negatives: 262.0000
Epoch 68/15000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6945 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 252.0000 - true_negatives: 259.0000 - false_positives: 252.0000 - false_negatives: 260.0000
Epoch 69/15000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6945 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 252.0000 - true_negatives: 259.0000 - false_positives: 252.0000 - false_negatives: 260.0000
Epoch 70/15000
1023/1023 [==============================] - 0s 4us/sample - loss: 0.6945 - categorical_accuracy: 1.0000 - accuracy: 0.0000e+00 - true_positives: 252.0000 - true_negatives: 257.0000 - false_positives: 254.0000 - false_negative

In [ ]:
plt.figure(figsize=(9.9,1))
plt.plot(Y[0:400],'.',c='k',markersize=2)
plt.plot((model.predict(X.astype('float64'))>=0.5)[0:400],c='r',alpha=0.5)
plt.xlim([0,400])
plt.tight_layout()

In [ ]:
###############################################################################
#  Map numeric neural network weights to normalized colormap values in [0,1]  #
###############################################################################

def num2cmap(x, max_magnitude, cmap, epsilon=1e-9):
    """
    DESCRIPTION:
    This function accepts a float value x and remaps it to a colorscale speci-
    fied via colormap parameter cmap. This function also accepts a limit on
    the expected value |x|--max_magnitude--which aids in normalization of the
    value x to the valid matplotlib colorscale specified by cmap (in string ID
    format). A stability/damping paramter epsilon is also provided to prevent
    against division by zero during the normalization of scalar/vector x which
    is required for cmap(), which itself outputs a 4D color vector per element.
    Thus, an array which has 1 color vector per element of x is output.

    INPUTS & OUTPUTS:
    :param x: input value to be remapped to some colormap defined by cmap
    :type x: float
    :param max_magnitude: max. |x| which could be expected by this function
    :type max_magnitude: float
    :param cmap: some valid matplotlib colormap string ID
    :type cmap: str
    :param epsilon: let epsilon > 0 be small (prevents division by zero)
    :type epsilon: float
    :returns: color vector on the color scale specified, proportional to input
    :rtype: numpy.ndarray
    """
    return cmap(0.5 + x / (epsilon + 2 * max_magnitude))

###############################################################################
#  Diagram an XOR connection between nodes [n, n + 1] in a Galois-style LFSR  #
###############################################################################

def lfsr_xor_connection(n, y_register=0, y_feedback=-0.1, lw=1, xor_size=8):
    """
    DESCRIPTION:
    This function draws the wiring between nodes / bits [n, n + 1] of a Galois
    style LFSR for the specific case where there is an XOR gate receiving 
    feedback tapped from the output of the register. The XOR gate is impro-
    vised from matplotlib markers. The wiring of the register is rerouted from
    its baseline value to form symmetric XOR gate inputs--1 from the n'th bit,
    1 from the feedback from the register output. The output of the register
    is then routed (to the right) to where the (n + 1)'st bit is positioned.
    This function is defined not only with respect to the y-coordinate of the
    LFSR state vector, but also that of the feedback loop (below it). The 
    register size and wiring width are also custom input parameters.

    INPUTS & OUTPUTS:
    :param n: XOR gate is at interval of register bits indexed [n, n + 1]
    :type n: int
    :param y_register: y-coordinate of the register along plot vertical axis
    :type y_register: float
    :param y_feedback: y-coordinate of the feedback loop along vertical axis
    :type y_feedback: float
    :param lw: line width of XOR wiring in plot/diagram (FWDed to matplotlib)
    :type lw: int
    :param xor_size: size of the XOR gate icon improvised from plot markers
    :type xor_size: int
    :returns: nothing (this function only plots things)
    :rtype: None
    """
    #############
    # XOR input #
    #############

    # Wire y-offset into XOR gate
    dy = 0.005

    # Verticle register wiring offset into XOR gate
    plt.plot([n + 0.2, n + 0.2], [-0.1, y_register - dy], lw=1, c='k')

    # Vertically displaced [horiz.] wire feeding XOR gate via tap
    plt.plot([n + 0.2, n + 0.55], [y_register - dy,
        y_register - dy], lw=lw, c='k')

    # Vertically displaced [horiz.] wire feeding XOR gate via register
    plt.plot([n, n + 0.55], [y_register + dy, y_register + dy], lw=1, c='k')

    ############
    # XOR gate #
    ############
    
    # XOR gate contribution from '|' marker
    plt.plot([n + 0.55], [y_register],
        '|', c='k', lw=lw, markersize=xor_size)
    
    # XOR gate contribution from '>' marker
    plt.plot([n + 0.67], [y_register], 
        '>', c='k', lw=lw, markersize=(xor_size - 1))

    ##############
    # XOR output #
    ##############

    # Output wire from XOR gate (level w/ register bits)
    plt.plot([n + 0.6, n + 1], [y_register, y_register], c='k', lw=lw)

###############################################################################
#  Diagram a shift (>>) connection between bits [n, n + 1] in a Galois LFSR   #
###############################################################################

def lfsr_shift_connection(n, y_register=0, lw=1, arrow_size=8):
    """
    """
    # Actual wiring between bits [n, n + 1]
    plt.plot([n, n + 1], [y_register, y_register], c='k', lw=lw)

    # Shift operator on wire b/w bits [n, n + 1]
    plt.plot([n + 0.5], [y_register], '4', c='k',
        lw=lw, markersize=arrow_size)

###############################################################################
#  Initialize the input nodes (aka. the LFSR state vector) of the neural net  #
###############################################################################

def init_input_nodes(deg, seed_bits, markersize=8):
    """
    """
    # For each n'th bit...
    for n in range(deg):

        # Assign black node for bit=1, white for bit=0
        nth_node_color = 'k' if seed_bits[n] else 'w'

        ########################################
        # Plot LFSR n'th input node of network #
        ########################################

        # Plot the n'th node in the register seed state
        plt.plot([n],[0],'o', markersize=markersize, 
            markeredgecolor='k', 
            markerfacecolor=nth_node_color);

###############################################################################
#  Initialize the hidden nodes (2nd layer) of the feedforward neural network  #
###############################################################################

def init_hidden_nodes(deg, y_layer=0.5, markersize=8):
    """
    """
    # For each n'th node...
    for n in range(deg):

        # Plot hidden node @ coordinate (n,y_layer)
        plt.plot([n], [y_layer], 'o', markersize=markersize,
            markeredgecolor='k', markerfacecolor='w');

###############################################################################
#  Initialize the network output/activation node & decision/prediction node   #
###############################################################################

def init_decision_nodes(x_node, y_lower=0.8, y_upper=1, markersize=8):
    """
    """
    # Sigmoidal class prediction node
    plt.plot([x_node],[y_lower],'o',
        markersize=markersize,
        markeredgecolor='k',
        markerfacecolor='w');

    # Thresholded bit prediction node
    plt.plot([x_node],[y_upper],'o',
        markersize=markersize,
        markeredgecolor='k',
        markerfacecolor='w');

###############################################################################
# Given a colormap & set of model weights, map model weights to color values  #
###############################################################################
 
def network_weight_coloration(model, colormap='twilight_shifted'):
    """
    """
    # Max. weight modulus across network
    max_weight = max([
        max(model.weights[0].numpy()), # Layer 0 (input)
        max(model.weights[1].numpy()) # Layer 1 (hidden)
    ])

    # Define colormap via matplotlib
    cmap = plt.get_cmap(colormap)

    # Color values for weights of Layer 0
    layer0_colors = num2cmap(
        model.weights[0].numpy(),
        max_magnitude=max_weight,
        cmap=cmap)

    # Color values for weights of Layer 1
    layer1_colors = num2cmap(
        model.weights[1].numpy(),
        max_magnitude=max_weight,
        cmap=cmap)
    
    # Return color arrays (w/ depth = 4)
    return layer0_colors, layer1_colors

###############################################################################
#  Draw register tap wiring/gates in the LFSR diagram beneath the neural net  #
###############################################################################

def lfsr_polynomial_wiring(deg, taps, y_register=0,
    y_feedback=-0.1, lw=1, arrow_size=8, xor_size=8):
    """
    """
    # For each n'th polynomial coeff...
    for n in range(deg - 1):
        
        #######################################
        # Decorate wiring b/w bits [n, n + 1] #
        #######################################

        # If nonzero n'th coefficient...
        if n > 0 and taps[n]:

            # Place X0R gate w/ feedback taps
            lfsr_xor_connection(n, y_register=y_register, 
                y_feedback=y_feedback, lw=lw, xor_size=xor_size)

        # n'th coefficient is 0...
        else:

            # Place ordinary shift operator
            lfsr_shift_connection(n, y_register=y_register, 
                lw=lw, arrow_size=arrow_size)

###############################################################################
#  Draw the feedback loop wiring in the LFSR diagram beneath the neural net   #
###############################################################################

def lfsr_feedback_loop_wiring(x_midpt, deg,
    y_register=0, y_feedback=-0.1, lw=1, arrow_size=8):
    """
    """
    ##############################################
    # Register I/O wiring (level with LFSR bits) #
    ##############################################

    # Input wire (level with register)
    plt.plot([0, -0.5], [y_register, y_register], c='k', lw=lw)

    # Shift operator decorating input wire
    plt.plot([-0.25], [y_register], '4', c='k',
        markersize=arrow_size)

    # Output wire (level with register)
    plt.plot([deg - 1, deg - 0.5], 
        [y_register, y_register],
        c='k', lw=lw)

    ###################################
    # Descending feedback loop wiring #
    ###################################

    # LFSR output wire, descending into feedback loop
    plt.plot([deg - 0.5, deg - 0.5], [y_register, y_feedback],
        c='k', lw=lw)

    # Shift operator decorating feedback loop wire
    plt.plot([deg - 0.5], [(y_register + y_feedback) / 2],
        '1', c='k', markersize=arrow_size)

    ##################################
    # Ascending feedback loop wiring #
    ##################################

    # LFSR input wire, ascending out of feedback loop
    plt.plot([-0.5, -0.5], [y_register, y_feedback], c='k', lw=lw)

    # Shift operator decorating feedback loop wire
    plt.plot([-0.5],[(y_register + y_feedback) / 2],
        '2',c='k', markersize=arrow_size)

    ##############################
    # Lower feedback loop wiring #
    ##############################

    # Lower LFSR feedback loop wire (spans register length)
    plt.plot([-0.5, deg - 0.5], [y_feedback, y_feedback], c='k', lw=lw)

    # Shift operator decorating feedback loop wire
    plt.plot([x_midpt], [y_feedback], '3', c='k', markersize=arrow_size)

###############################################################################
#  Diagram a Galois-style linear feedback shift register (LFSR), given taps   #
###############################################################################

def draw_lfsr(taps, deg, x_midpt, y_register=0, 
    y_feedback=-0.1, arrow_size=8, lw=1):
    """
    """
    # Within-register wiring
    lfsr_polynomial_wiring(deg,
        taps, y_register=y_register, 
        lw=lw, arrow_size=arrow_size)

    # Feedback loop wiring
    lfsr_feedback_loop_wiring(
        x_midpt, deg=deg,
        y_register=y_register,
        y_feedback=y_feedback,
        lw=lw, arrow_size=arrow_size)

###############################################################################
# Draw the linkage between the network output node & predicted bit (decision) #
###############################################################################

def draw_prediction_link(x_midpt, y_lower=0.8, y_upper=1, lw=1):
    """
    """
    # Edge connecting sigmoid node to thresholded bit prediction
    plt.plot([x_midpt, x_midpt], [y_lower, y_upper], c='k', lw=lw)

###############################################################################
#    Draw weighted network edges of layer 0 of the feedforward neural net     #
###############################################################################

def draw_layer0_linkages(colors, num_input, num_hidden,
    y_hidden=0.5, alpha=0.75, lw=1):
    """
    """
    # For each n'th node in input layer...
    for n in range(num_input):

        # For each m'th node in hidden layer...
        for m in range(num_hidden):

            # Draw link between n'th input & m'th hidden nodes
            plt.plot([n, m], [0, y_hidden], c=colors[n, m, :], 
                alpha=alpha, lw=lw)

###############################################################################
#    Draw weighted network edges of layer 1 of the feedforward neural net     #
###############################################################################

def draw_layer1_linkages(colors, num_hidden, x_midpt, 
    y_hidden=0.5, y_output=0.8, alpha=0.75, lw=1):
    """
    """
    # For each n'th hidden layer node...
    for n in range(num_hidden):
        
        # Output layer's link from n'th hidden neuron
        plt.plot([n, x_midpt], [y_hidden, y_output], 
            c=colors[n, 0, :], alpha=alpha, lw=lw)

###############################################################################
# Draw an imagined feedback connection from the network output to LFSR input  #
###############################################################################

def imagine_feedback_from_network(x_midpt,y_upper=1, y_lower=0,
    y_midpt=-0.5, arrow_size=8, lw=1, linestyle='--'):
    """
    """
    #################################
    # Horizontal feedback loop wire #
    #################################
    
    # Horizontal wire at top of feedback loop
    plt.plot([y_midpt, x_midpt], 
             [y_upper, y_upper], 
             linestyle, c='k', lw=lw)
    
    # Place feedback arrow in center of wire
    plt.plot([x_midpt / 2 - 0.25], [y_upper],
        '3', c='k', markersize=arrow_size)
    
    ###############################
    # Vertical feedback loop wire #
    ###############################
    
    # Descending wire at lefthand side of feedback loop
    plt.plot([-0.5, -0.5], [y_upper, y_lower],
        linestyle, c='k', lw=lw)

    plt.plot([-0.5], [(y_upper + y_lower) / 2],
        '1', c='k', markersize=arrow_size)

###############################################################################
# Draw the wiring of the 2-layer feedforward neural network learning the LFSR #
###############################################################################

def draw_network_wiring(model, config):
    """
    """
    # Map neural network weights to color scale
    c_w0, c_w1 = network_weight_coloration(
        model, colormap=config['cmap'])

    # Plot network weighted edges from input layer to
    # hidden layer, mapping weight values to colors
    draw_layer0_linkages(c_w0, 
        num_input=config['deg'],
        num_hidden=config['deg'], 
        y_hidden=config['y_hidden'], 
        alpha=config['net_link_alpha'],
        lw=config['link_width'])

    # Plot network weighted edges from hidden layer to
    # output layer, mapping weight values to colors
    draw_layer1_linkages(c_w1,
        num_hidden=config['deg'], 
        x_midpt=config['x_mid'],
        y_hidden=config['y_hidden'], 
        y_output=config['y_output'], 
        alpha=config['net_link_alpha'], 
        lw=config['link_width'])

    # Edge connecting sigmoid node to thresholded bit prediction
    draw_prediction_link(x_midpt=config['x_mid'], 
        y_lower=config['y_output'],
        y_upper=config['y_decision'],
        lw=config['link_width'])

###############################################################################
#  Initialize feedforward neural network diagram, populating nodes and edges  #
###############################################################################

def init_network_diagram(model, config, seed_bits):
    """
    """
    #########
    # Edges #
    #########

    # Draw the network connections/edges
    draw_network_wiring(model, config)

    # Draw imagined/pretend feedback from network output to LFSR
    imagine_feedback_from_network(
        x_midpt=config['x_mid'],
        y_upper=config['y_decision'],
        y_lower=config['y_input'],
        arrow_size=config['arrow_size'],
        lw=config['link_width'],
        linestyle=config['net_feedback_linestyle'])

    #########
    # Nodes #
    #########

    # Input LFSR nodes / bits (deg total)
    init_input_nodes(
        deg=config['deg'], 
        seed_bits=seed_bits,
        markersize=config['node_size'])

    # Hidden layer nodes (deg total)
    init_hidden_nodes(
        deg=config['deg'],
        y_layer=config['y_hidden'],
        markersize=config['node_size'])

    # Output probability + decision nodes
    init_decision_nodes(
        x_node=config['x_mid'],
        y_lower=config['y_output'],
        y_upper=config['y_decision'],
        markersize=config['node_size'])

In [ ]:
from numpy import max

config = {
    'figsize': (9.9, 6),
    'deg': deg,
    'x_mid': (deg - 1) / 2,
    'cmap': 'plasma',#'twilight_shifted',
    'y_input': 0,
    'y_hidden': 0.5,
    'y_output': 0.8,
    'y_decision': 1,
    'y_LFSR_loop': -0.1,
    'y_lim': [-0.2, 1.1],
    'num_hidden': deg,
    'num_input': deg,
    'node_size': 8,
    'link_width': 1,
    'net_link_alpha': 0.75,
    'arrow_size': 8,
    'net_feedback_linestyle':'--'
}

#############################
# Set up shop in new figure #
#############################

# New figure
plt.figure(figsize=config['figsize'])

# Draw LFSR corrsponding to tap polynomial
draw_lfsr(taps=str2vec(hex2bin(
    coeff_catalog[p], config['deg'])), 
    deg=config['deg'], 
    x_midpt=config['x_mid'],
    y_register=config['y_input'],
    y_feedback=config['y_LFSR_loop'],
    arrow_size=config['arrow_size'],
    lw=config['link_width'])

# Initialize neural net drawing
init_network_diagram(model=model,
    config=config, seed_bits=X[0,:])

###############
# Format axes #
###############

# Vertical axis span
plt.ylim(config['y_lim']);

# Remove all axis ticks
plt.xticks([]);
plt.yticks([]);